<a href="https://colab.research.google.com/github/msubhasish789/machineLearning/blob/main/Basic_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch transformers datasets scikit-learn pandas numpy matplotlib


In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split


In [ ]:
model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)  # Change num_labels based on your dataset


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Example: Load CSV file
df = pd.read_csv("/content/Word_2_আগ.csv")

# Ensure columns exist
df = df[['context', 'Word', 'label']]

# Convert to Hugging Face Dataset format
train_texts, val_texts, train_labels, val_labels = train_test_split(df['context'], df['label'], test_size=0.2)

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True)

train_dataset = Dataset.from_dict({"context": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"context": val_texts, "label": val_labels})

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/117 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


<ipython-input-12-2a67ae787459>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [2]:
import pandas as pd

In [3]:
df =  pd.read_csv("/content/Word_2_আগ.csv")

In [4]:
df.head()

,Word,context,label,English Translation,Google Translation,Microsoft Bing Translation
0,আগ,প্রাত্যহিক জীবনে আগ অত্যন্ত উপযোগী ও এর গুরুত্...,আগুন,"In daily life, fire is extremely useful, and i...",Aga is very useful in daily life and its impor...,Ag is very useful in daily life and its import...
1,আগ,তাই আগ ব্যবহারে অত্যন্ত সচেতনতার পরিচয় দিতে হয়।,আগুন,"Therefore, one must exercise utmost caution wh...","Therefore, extreme caution must be exercised i...","Therefore, it is necessary to introduce great ..."
2,আগ,ঠাণ্ডা কিংবা শীতকালে অগ্নি প্রজ্জ্বলনের মাধ্যম...,আগুন,"In cold or winter seasons, fires are lit to ke...","During cold or winter, houses are kept warm by...","In cold or winter, the house is kept warm by l..."
3,আগ,সর্বোপরি তাপ শক্তির সাহায্যে রান্নাকার্য পরিচা...,আগুন,"Above all, thermal energy is used to operate c...","Above all, it greatly controls human life acti...","Above all, it greatly controls the activities ..."
4,আগ,সাবধানতার সাথে আগ না জ্বালালে এটি মারাত্মক প্র...,আগুন,"If fire is not lit with caution, it can cause ...",It can cause serious reactions and effects if ...,It causes serious repercussions and effects if...


In [6]:
df.isnull().sum()

,0
Word,0
context,0
label,0
English Translation,0
Google Translation,0
Microsoft Bing Translation,0


In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>|,'), '', raw_text)
    return cleaned_text

In [9]:
df['context'] = df['context'].apply(remove_tags)

In [10]:
df.head()

,Word,context,label,English Translation,Google Translation,Microsoft Bing Translation
0,আগ,প্রাত্যহিক জীবনে আগ অত্যন্ত উপযোগী ও এর গুরুত্...,আগুন,"In daily life, fire is extremely useful, and i...",Aga is very useful in daily life and its impor...,Ag is very useful in daily life and its import...
1,আগ,তাই আগ ব্যবহারে অত্যন্ত সচেতনতার পরিচয় দিতে হয়।,আগুন,"Therefore, one must exercise utmost caution wh...","Therefore, extreme caution must be exercised i...","Therefore, it is necessary to introduce great ..."
2,আগ,ঠাণ্ডা কিংবা শীতকালে অগ্নি প্রজ্জ্বলনের মাধ্যম...,আগুন,"In cold or winter seasons, fires are lit to ke...","During cold or winter, houses are kept warm by...","In cold or winter, the house is kept warm by l..."
3,আগ,সর্বোপরি তাপ শক্তির সাহায্যে রান্নাকার্য পরিচা...,আগুন,"Above all, thermal energy is used to operate c...","Above all, it greatly controls human life acti...","Above all, it greatly controls the activities ..."
4,আগ,সাবধানতার সাথে আগ না জ্বালালে এটি মারাত্মক প্র...,আগুন,"If fire is not lit with caution, it can cause ...",It can cause serious reactions and effects if ...,It causes serious repercussions and effects if...


In [14]:
import requests

# URL of the stopwords dataset
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-bn/master/stopwords-bn.txt"

# Fetch and load Bengali stopwords
response = requests.get(url)
bengali_stopwords = set(response.text.split("\n"))  # Convert to a set for faster lookups



In [15]:
bengali_stopwords

{'অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [16]:
df['cleaned_context'] = df['context'].apply(lambda x: " ".join([word for word in x.split() if word not in bengali_stopwords]))


In [17]:
df.head()

,Word,context,label,English Translation,Google Translation,Microsoft Bing Translation,cleaned_context
0,আগ,প্রাত্যহিক জীবনে আগ অত্যন্ত উপযোগী ও এর গুরুত্...,আগুন,"In daily life, fire is extremely useful, and i...",Aga is very useful in daily life and its impor...,Ag is very useful in daily life and its import...,প্রাত্যহিক জীবনে আগ অত্যন্ত উপযোগী গুরুত্ব অপর...
1,আগ,তাই আগ ব্যবহারে অত্যন্ত সচেতনতার পরিচয় দিতে হয়।,আগুন,"Therefore, one must exercise utmost caution wh...","Therefore, extreme caution must be exercised i...","Therefore, it is necessary to introduce great ...",আগ ব্যবহারে অত্যন্ত সচেতনতার পরিচয় হয়।
2,আগ,ঠাণ্ডা কিংবা শীতকালে অগ্নি প্রজ্জ্বলনের মাধ্যম...,আগুন,"In cold or winter seasons, fires are lit to ke...","During cold or winter, houses are kept warm by...","In cold or winter, the house is kept warm by l...",ঠাণ্ডা শীতকালে অগ্নি প্রজ্জ্বলনের ঘর গরম অন্ধক...
3,আগ,সর্বোপরি তাপ শক্তির সাহায্যে রান্নাকার্য পরিচা...,আগুন,"Above all, thermal energy is used to operate c...","Above all, it greatly controls human life acti...","Above all, it greatly controls the activities ...",সর্বোপরি তাপ শক্তির সাহায্যে রান্নাকার্য পরিচা...
4,আগ,সাবধানতার সাথে আগ না জ্বালালে এটি মারাত্মক প্র...,আগুন,"If fire is not lit with caution, it can cause ...",It can cause serious reactions and effects if ...,It causes serious repercussions and effects if...,সাবধানতার সাথে আগ জ্বালালে মারাত্মক প্রতিক্রিয...


In [43]:
X = df.loc[:, ["cleaned_context"]]
y=df['label']

In [44]:
X


,cleaned_context
0,প্রাত্যহিক জীবনে আগ অত্যন্ত উপযোগী গুরুত্ব অপর...
1,আগ ব্যবহারে অত্যন্ত সচেতনতার পরিচয় হয়।
2,ঠাণ্ডা শীতকালে অগ্নি প্রজ্জ্বলনের ঘর গরম অন্ধক...
3,সর্বোপরি তাপ শক্তির সাহায্যে রান্নাকার্য পরিচা...
4,সাবধানতার সাথে আগ জ্বালালে মারাত্মক প্রতিক্রিয...
...,...
142,চুলের আগ ফাটা রোধ ঘরোয়া উপায়ে।
143,মেয়েদের সৌন্দর্যের গুরুত্বপূর্ণ অংশ চুল।
144,চুল রুক্ষ প্রাণহীন পড়লে চেহারাও মলিন পড়ে।
145,চুলের আগ ফেটে চুল স্বাভাবিক জৌলুস হারিয়ে রুক্...


In [45]:
y

,label
0,আগুন
1,আগুন
2,আগুন
3,আগুন
4,আগুন
...,...
142,অগ্রভাগ
143,অগ্রভাগ
144,অগ্রভাগ
145,অগ্রভাগ


In [46]:
y.value_counts()

,count
label,
অগ্রভাগ,59
ওহে,47
আগুন,41


In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [48]:
X_train.shape

(117, 1)

In [30]:
# import numpy as np
# X_train = np.array(X_train).reshape(-1, 1)

In [36]:
# X_train.shape

(117, 1)

In [49]:
X_train.head()

,cleaned_context
125,ভাল পণ্য কিনতে পাইকারি নিশ্চিত করুন বিনামূল্যে...
132,অতিরিক্ত শ্যাম্পু শ্যাম্পুর রাসায়নিক উপাদানও ...
42,তিন মাসের বাড়িভাড়া পাওনা?
138,চুলের সম্যসাটায় বেশিরভাগ মেয়েরা ভুগে চুলের আ...
28,মুহূর্তের গুদামে ছড়িয়ে পড়ে আগ।


In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
cv = CountVectorizer()

In [50]:
X_train_bow = cv.fit_transform(X_train['cleaned_context']).toarray()
X_test_bow = cv.transform(X_test['cleaned_context']).toarray()

In [51]:
X_train_bow.shape

(117, 257)

## naive_bayes GaussianNB

In [64]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [65]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6333333333333333

In [67]:
inputText = input("Enter your text input : ")
data = cv.transform([inputText]).toarray()
output =  gnb.predict(data)
print("Your Entered Language is :", output)

Enter your text input : আগে গেলে দেখতে পাবে 
Your Entered Language is : ['ওহে']


# RandomForest

In [54]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.5666666666666667

In [55]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['cleaned_context']).toarray()
X_test_bow = cv.transform(X_test['cleaned_context']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)


0.5666666666666667

In [57]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['cleaned_context']).toarray()
X_test_bow = cv.transform(X_test['cleaned_context']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.5333333333333333

# TF-IDF


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
tfidf = TfidfVectorizer()

In [61]:
X_train_tfidf = tfidf.fit_transform(X_train['cleaned_context']).toarray()
X_test_tfidf = tfidf.transform(X_test['cleaned_context']).toarray()

In [62]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)


0.6